In [ ]:
#model parameters
import tensorflow as tf
import numpy as np
import samples as ls
import model_types
import os

step_counter = 0
create_model = True
load_dnn = True #ignored when create_model is set
load_samples = False
model_num = 2
model_type = 11
smooth_steps = 0

model_id = '%s_%s' % (model_type, model_num)
labels,sample_ids,filename = model_types.get_labels(model_type)
model_path = 'tmp/dnn/model/dnn-%s' % model_id

if model_num == 0:
    accel,gyro,compass = 'xyz','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [50,10]
elif model_num == 1:
    accel,gyro,compass = 'xyz','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [50]
elif model_num == 2:
    accel,gyro,compass = 'xy','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [20,10]
elif model_num == 3:
    accel,gyro,compass = 'xy','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [20]
elif model_num == 4:
    accel,gyro,compass = 'xy','z',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [10]
elif model_num == 5:
    accel,gyro,compass = 'xy','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [10,10]
elif model_num == 6:
    accel,gyro,compass = 'xy','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    hidden_layers = [120,50]
elif model_num == 7:
    accel,gyro,compass = 'xy','xyz',''
    input_dim = len(accel)+len(gyro)+len(compass)
    smooth_steps = 3
    hidden_layers = [120,50]
else:
    raise ValueError('Unknown model')
    

if create_model:
    if not os.path.exists(model_path):
        load_dnn = False
        os.makedirs(model_path)
    else:
        load_dnn = True

classes = len(labels)

hidden_layers.insert(0,(50-(smooth_steps*2))*input_dim)
hidden_layers.append(classes)

#percentage used for test data
test_rate = 0.1

learning_rate = 0.01

In [ ]:
#load samples
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)

    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)
        
    samples.convert_to_input(size = None, accel=accel, gyro=gyro, compass=compass)
    
    ls.save(samples, filename)
    
samples.unroll(steps=50) #fixed for angle samples at 50 values
samples.convert_to_onehot()
for i in range(smooth_steps): samples.smooth()
samples.split_test(test_rate)
samples.flatten()

In [ ]:
#create graph
if not load_dnn:
    tf.logging.set_verbosity(tf.logging.ERROR)
    tf.reset_default_graph()

    xtest,ytest,_ = samples.get_test(padding=False)
    xtest = np.array(xtest)
    ytest = np.array(ytest)


    x = tf.placeholder(tf.float32, [None, hidden_layers[0]], name='x')
    y = tf.placeholder(tf.int64, [None, classes], name='y')

    Ws = []
    for i in range(len(hidden_layers)-1):
        W = tf.Variable(tf.random_normal([hidden_layers[i], hidden_layers[i+1]]))
        b = tf.Variable(tf.zeros(hidden_layers[i+1]))
        Ws.append((W,b))


    preactivations = []
    activations = []
    act = x
    for W,b in Ws:
        preact = tf.matmul(act, W) + b
        preactivations.append(preact)
        act = tf.sigmoid(preact)
        activations.append(act)
    dnn = tf.nn.softmax(preact)
    dnn_pred = tf.argmax(dnn, 1)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=preact))
    correct_prediction = tf.equal(tf.argmax(y,1), dnn_pred)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)


In [ ]:
#save and load graph
sess = tf.Session()

if load_dnn:
    saver = tf.train.import_meta_graph('%s/dnn-%s.meta' % (model_path,model_id))
    saver.restore(sess, tf.train.latest_checkpoint(model_path))
    optimizer = tf.get_collection('optimizer')[0]
    accuracy = tf.get_collection('accuracy')[0]
    dnn_pred = tf.get_collection('dnn_pred')[0]
    dnn = tf.get_collection('dnn')[0]
    merged = tf.get_collection('merged')[0]
    x = tf.get_collection('x')[0]
    y = tf.get_collection('y')[0]
    train_writer = tf.summary.FileWriter('tmp/dnn/summary-%s' % model_id, sess.graph)
else:
    saver = tf.train.Saver()

    #summaries
    def var_summary(var, name):
        mean = tf.reduce_mean(var)
        tf.summary.scalar(name + '_mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar(name + '_stddev', stddev)
        tf.summary.scalar(name + '_max', tf.reduce_max(var))
        tf.summary.scalar(name + '_min', tf.reduce_min(var))
        tf.summary.histogram(name + '_histogram', var)
    i = 0
    for W,b in Ws: 
        var_summary(W, 'weights_%s' % i)
        var_summary(b, 'bias_%s' % i)
        i += 1
    for i in range(len(activations)):
        tf.summary.histogram('preactivations_%s' % i, preactivations[i])
        tf.summary.histogram('activations_%s' % i, activations[i])
    tf.summary.scalar('loss', loss)

    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('tmp/dnn/summary-%s' % model_id, sess.graph)

    #init or load from checkpoint
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tf.add_to_collection('optimizer', optimizer)
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('dnn_pred', dnn_pred)
    tf.add_to_collection('merged', merged)
    tf.add_to_collection('x', x)
    tf.add_to_collection('y', y)
    tf.add_to_collection('dnn', dnn)
    saver.save(sess, '%s/dnn-%s' % (model_path,model_id))
    
    

In [ ]:
#convert label class to angle floats

def test_all():
    xtest,ytest,_ = samples.get_all(use_labels_data=False)
    acc = sess.run(accuracy, feed_dict={x: xtest, y: ytest})
    print("Accuracy train: %s" % acc)
    return acc
    
def test():
    xtest,ytest,_ = samples.get_test(use_labels_data=False)
    acc = sess.run(accuracy, feed_dict={x: xtest, y: ytest})
    print("Accuracy test: %s" % acc)
    return acc
    
def test_total():
    acc1 = test_all()
    acc2 = test()
    acc = acc1*(1.0-test_rate) + acc2*test_rate
    
    print('Accuracy total: %s' % acc)
    return acc

def train(steps, batch_size):
    global step_counter
    progress = 5
    for i in range(steps):
        xtrain,ytrain,_ = samples.get_batch(batch_size, use_labels_data=False)
        summary,_ = sess.run([merged, optimizer], feed_dict={x: xtrain, y: ytrain})
        train_writer.add_summary(summary, i+step_counter)
        if i>=progress/100.0*steps-1:
            print("Progress: %s%%" % progress)
            progress += 5
            test()
            saver.save(sess, '%s/dnn-%s' % (model_path,model_id), global_step=step_counter)
    step_counter += steps
    saver.save(sess, '%s/dnn-%s' % (model_path,model_id), global_step=step_counter)
    
def per_label_acc(test=True):
    if test:
        xtest,ytest,seq = samples.get_test(use_labels_data=False)
    else:
        xtest,ytest,seq = samples.get_all(use_labels_data=False)
    accs = [0] * classes
    counts = [0] * classes

    for i in range(len(ytest)):
        acc = sess.run(accuracy, feed_dict={x: [xtest[i]], y: [ytest[i]]})
        c = 0
        for k in range(len(ytest[i])):
            if ytest[i][k] > 0.5:
                c = k
                break
        accs[c] += acc
        counts[c] += 1

    for i in range(classes):
        acc = accs[i]/counts[i]
        print(counts[i])
        print('%s: %s' % (labels[i],acc))
        
def predict(x_input):
    return sess.run(dnn_pred, feed_dict={x:x_input})


In [ ]:
if True:
    for key in samples.data:
        print('%s: %s' % (key, len(samples.data[key])))
        #print('%s[0]: %s' % (key, len(samples.data[key][0])))
        
    xtrain,ytrain,_ = samples.get_batch(10, use_labels_data=False)
    res = sess.run([dnn], feed_dict={x: [xtrain[0]], y: [ytrain[0]]})
    print(res)
    print(sum(res[0][0]))

In [ ]:
if True:
    train(10000,40)
    test_total()

In [ ]:
if True:
    per_label_acc()